In [1]:
!pip install langchain==0.0.354
!pip install pgvector

In [2]:
!pip install pypdf

# importing the libraries

In [3]:
import json
import os
import sys

import boto3
import pandas as pd
import numpy as np
from io import StringIO
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
import os
import psycopg2
from langchain.vectorstores.pgvector import PGVector
from langchain_community.vectorstores.pgvector import PGVector
import datetime


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


# queries to be asked

In [4]:
# New query list
query_list=[
 "What is the Intimation date period?",
"what is Confirmation Date time period?",
"From when does the credit terms applicable for making the payments?",
 "Who has ownership of materials license?",
"what about Audit rights?",
"what is period of Reconciliation of Payments",
"how much is Term and Termination period?", 
"Which court has jurisdiction over disputes under the agreement?",
"what is Insurance policy in the agreement?",
"how much is time period for Shipping Confirmation?",
"what are Receivables policy?",
"how much Credit Note time period?",
"is there any provision for debit note? explain:",
"who is liable for indirect damages?",
"are there any provisions or clauses for indemnification?",
"what about TITLE TO AND RISK OF LOSS OF THE PRODUCTS?",
"what is Resolution Date time period?",
"how much time is for Returnable Products?",
"what is policy for  Product Returns / Returnable Products?",
"are there any provisions or clauses for Warranties and Covenants?",
"what is time period for MSME?",
"what does agreements tells about Denied Tax Credits?",
"who communicate the format of  Compliant Invoice?",
"is PO defined in the agreement?",
"What are the brands mentioned in the agreement?"
]

In [5]:
query_list=["Which court has jurisdiction over disputes under the agreement?"]

# Combined code for getting output

In [6]:
# creating the bedrock client
bedrock_client = boto3.client('bedrock-runtime',region_name='us-east-1',aws_access_key_id='',aws_secret_access_key='')
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)


# connecting to postgre database
DRIVER=os.environ.get("PGVECTOR_DRIVER", "psycopg2")
HOST=os.environ.get("PGVECTOR_HOST", "database-1.c5kqyi6ckky7.ap-south-1.rds.amazonaws.com")
PORT=os.environ.get("PGVECTOR_PORT", "5432")
DATABASE=os.environ.get("PGVECTOR_DATABASE", "postgres")
USER=os.environ.get("PGVECTOR_USER", "postgres")
PASSWORD=os.environ.get("PGVECTOR_PASSWORD", "bedrock123")


conn = psycopg2.connect(dbname=DATABASE,user=USER,host=HOST,password=PASSWORD)
cur = conn.cursor()
cur.execute('CREATE EXTENSION IF NOT EXISTS vector;')
conn.commit()

CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver=DRIVER,
    host=HOST,
    port=PORT,
    database=DATABASE,
    user=USER,
    password=PASSWORD
)
def split_doc_chunks(document_name):    
    loader = PyPDFLoader(document_name)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        
        chunk_size=750,
        chunk_overlap=100,
    )
    docs = text_splitter.split_documents(documents)
    return docs
def collection(COLLECTION_NAME,docs):
    
    db = PGVector.from_documents(
    embedding=bedrock_embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING)

    vectorstore = PGVector(
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    embedding_function=bedrock_embeddings,)

    return vectorstore

def result_query_llama(docs, query,vector_store): 

    list=vector_store.similarity_search(query, k=7)
    
    context=""
    for i in range(0,len(list)):
        text=list[i].page_content
        context=context+text
    question = query
    prompt_data = f"""Answer the question in one line based only on the information provided in the context.If you don't know the answer, just say that you don't know, don't try to make up an answer.

    {context}
    #

    Question: {question}
    Answer:"""    
    
    
    body = json.dumps({
        "prompt": prompt_data,
        "max_gen_len": 256,
        "temperature": 0,
        "top_p": 0.1,
    })

    modelId = 'meta.llama2-13b-chat-v1'
    accept = 'application/json'
    contentType = 'application/json'

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

    response_body = json.loads(response.get('body').read())
    result_answer=response_body.get('generation')
    
    
    print(result_answer)
    return result_answer

def query_results(docs,vector_store):
    ans_list=[]
    df=pd.DataFrame(columns=["QUERY","ANSWER"])
    for query in query_list:
        ans_list.append(result_query_llama(docs, query,vector_store))
#         print(datetime.datetime.now())
    df["QUERY"]=query_list
        
    df["ANSWER"]=ans_list
    return df

def get_results(document_name):
    object_name=document_name.split(".")[0]+"llama2"
    docs=split_doc_chunks(document_name)
    
    vector_store=collection(object_name,docs)
#     result_answer=result_query(query)
    df=query_results(docs,vector_store)
    return df



In [7]:
document_name="Vendor_Agreement-Good_Luck_Footwears.pdf"

In [8]:
df=get_results(document_name)

 The agreement does not specify which court has jurisdiction over disputes under the agreement.


# save to s3 bucket

In [ ]:
output_s3_bucket='test-bucket-pt'
output_name=document_name.split(".")[0]+"pt"+"_llama2_pt_new1_final"+".csv"

# Save output data into s3 in csv format
csv_buffer = StringIO()
df.to_csv(csv_buffer,index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object(output_s3_bucket, output_name).put(Body=csv_buffer.getvalue())